# Imports

In [3]:
import requests 
from bs4 import BeautifulSoup
import csv 
from selenium import webdriver
import time
import sys
import argparse

# Scraper.PY

In [22]:
!python scraper.py -i --url "https://www.tripadvisor.com/Restaurants-g186338-London_England.html"

^C


# GithHub

In [ ]:
!git pull

In [7]:
!git add .
!git commit -m 
!git push origin main

[main e085330] Adding Cyprien'Notebook
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 chromedriver.exe


To https://github.com/Shins0o/nlp-project
   a7ce22e..e085330  main -> main


# Scrapping

In [ ]:
url = "https://uk.trustpilot.com/categories/restaurants_bars?location=London&trustscore=4.5&verified=true"

In [ ]:
def scrapeRestaurantsUrls(tripURLs):
    urls =[]
    for url in tripURLs:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.find('div', class_='_1kXteagE')
        stores = results.find_all('div', class_='wQjYiB7z')
        for store in stores:
            unModifiedUrl = str(store.find('a', href=True)['href'])
            urls.append('https://www.tripadvisor.com'+unModifiedUrl)            
    return urls


In [6]:
def scrapeRestaurantsUrls(url):
    print("entering first function")
    urls =[]
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    # print(soup)
    # results = soup.find('div', class_='_1kXteagE')
    stores = soup.find_all('div', class_='wQjYiB7z')
    for store in stores:
        unModifiedUrl = str(store.find('a', href=True)['href'])
        urls.append('https://www.tripadvisor.com'+unModifiedUrl)  
    print('urls : ',urls)          
    return urls

def scrapeRestaurantInfo(url):
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    storeName = soup.find('h1', class_='_3a1XQ88S').text
    avgRating = soup.find('span', class_='r2Cf69qf').text.strip()
    storeAddress = soup.find('div', class_= '_2vbD36Hr _36TL14Jn').find('span', class_='_2saB_OSe').text.strip()
    noReviews = soup.find('a', class_='_10Iv7dOs').text.strip().split()[0]
    print(storeName)
    with open(pathToStoreInfo, mode='a', encoding="utf-8") as trip:
        data_writer = csv.writer(trip, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        data_writer.writerow([storeName, storeAddress, avgRating, noReviews])

In [7]:
startingurl = "https://www.tripadvisor.com/Restaurants-g186338-London_England.html"
driver = webdriver.Edge('msedgedriver.exe')
# driver.get(url)
print("starting")
urls = scrapeRestaurantsUrls(startingurl)
for url in urls:
    print(url)
    #if you want to scrape restaurants info
    if info == True:
        scrapeRestaurantInfo(url)

    nextPage = True
    while nextPage:
        #Requests
        driver.get(url)
        time.sleep(1)
        #Click More button
        more = driver.find_elements_by_xpath("//span[contains(text(),'More')]")
        for x in range(0,len(more)):
            try:
                driver.execute_script("arguments[0].click();", more[x])
                time.sleep(3)
            except:
                pass
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #Store name
        storeName = soup.find('h1', class_='_3a1XQ88S').text
        #Reviews
        results = soup.find('div', class_='listContainer hide-more-mobile')
        try:
            reviews = results.find_all('div', class_='prw_rup prw_reviews_review_resp')
        except Exception:
            continue
        #Export to csv
        try:
            with open(pathToReviews, mode='a', encoding="utf-8") as trip_data:
                data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                for review in reviews:
                    ratingDate = review.find('span', class_='ratingDate').get('title')
                    text_review = review.find('p', class_='partial_entry')
                    if len(text_review.contents) > 2:
                        reviewText = str(text_review.contents[0][:-3]) + ' ' + str(text_review.contents[1].text)
                    else:
                        reviewText = text_review.text
                    reviewerUsername = review.find('div', class_='info_text pointer_cursor')
                    reviewerUsername = reviewerUsername.select('div > div')[0].get_text(strip=True)
                    rating = review.find('div', class_='ui_column is-9').findChildren('span')
                    rating = str(rating[0]).split('_')[3].split('0')[0]
                    data_writer.writerow([storeName, reviewerUsername, ratingDate, reviewText, rating])
        except:
            pass
        #Go to next page if exists
        try:
            unModifiedUrl = str(soup.find('a', class_ = 'nav next ui_button primary',href=True)['href'])
            url = 'https://www.tripadvisor.com' + unModifiedUrl
        except:
            nextPage = False


<ipython-input-7-c767e84702da>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('msedgedriver.exe')


starting
entering first function
